In [ ]:
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#pip install lxml
#from lxml import objectify
#from os import listdir
import xml.etree.ElementTree as ET

In [ ]:
path = 'dataset/data_2/'
xml_in = 'dataset/annotations.xml'

In [ ]:
tree = ET.parse(xml_in)
root = tree.getroot()
root.tag

In [ ]:
labels = []
i = 0
for child in tree.getroot():
    if child.tag == 'image':
        img = cv.imread(path + 'images/' + child.attrib['name'], cv.IMREAD_GRAYSCALE)
        for polygon in child:
            points = [[int(float(p)) for p in e.split(',')] for e in polygon.attrib['points'].split(';')]
            points = np.array(points, dtype=np.int32)
            mask = np.zeros(img.shape[0:2], dtype=np.uint8)
            cv.drawContours(mask, [points], -1, (255, 255, 255), -1, cv.LINE_AA)
            masked_img = img & mask
            rect = cv.boundingRect(points)
            out = masked_img[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]
            labels.append(polygon[0].text)
            cv.imwrite(path + f'poly_crops/{str(i).zfill(5)}.png', out)
            i += 1
df = pd.DataFrame(labels)
pd.to_csv(path + 'poly_labels.csv', header=False, index=False)